In [ ]:
import numpy as np
import pandas as pd
import polarDensity_helper as pc
from pathlib import Path
import matplotlib.pyplot as plt
from enrichment_plotters import polar_plot
plt.rcParams['axes.grid'] = False 

In [ ]:

def get_helices(the_path):
    # Optional helix locations
    try:
        helices_lwr = np.loadtxt(the_path.joinpath("Protein_coords_lwr.dat"))
        helices_upr = np.loadtxt(the_path.joinpath("Protein_coords_upr.dat"))
    except FileNotFoundError:
        helices_lwr = None
        helices_upr = None
        print("Protein coordinates not found")

    return helices_upr, helices_lwr

def read_rep(file_list, chains_groups, leaflets=['low','upp'], enrich=True):
    # get files to use

    enrichments = pd.DataFrame(index=chains_groups, columns=leaflets)
    counts = pd.DataFrame(index=chains_groups, columns=leaflets)

    idx = 0
    for fl in file_list:
        filename = fl.name
        tmp_chain = filename.split('.')[0]
        tmp_nm = filename.split('.')[2]
        rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(fl)
        # This is a hack. The above part does not have a "flexible"
        # method to consider sim type (a, b ...)
        idx+=1
        toadd = np.loadtxt(fl, skiprows=1)
        toadd = toadd[:,3:-1]
        counts.at[tmp_chain,tmp_nm] = toadd
        enrichments.at[tmp_chain,tmp_nm] = pc._analysis_call_(fl, radius, dr, dth, frames, enrich=enrich)

    return counts, enrichments

In [ ]:
def get_file_list(root, lipids):
    file_list = []
    for lip in lipids:
        toadd = list(root.glob(f"{lip}.dat*avg.dat") )
        file_list = np.append(file_list,toadd)
    return file_list

# Important User Settings:

In [ ]:
lipids = ["POPG", "POPE"]
leaflets = ['low', 'upp']
root = Path("/home/ems363/Projects/ELIC_PE_PG/aggregated/")
replicas = ["rep1", "rep3", "liam"]

enrichments = None
counts = None

for rep in replicas:
    reppath = root.joinpath(rep)
    file_list = get_file_list(reppath, lipids)
    rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(file_list[0])
    cts, rich = read_rep(file_list, lipids, leaflets, enrich=True)
    if counts is None:
        counts = cts
        enrichments = rich
    else:
        counts = counts+cts
        enrichments = enrichments+rich
    helices_upr, helices_lwr = get_helices(reppath)

counts = counts/len(replicas)
enrichments = enrichments/len(replicas)

In [ ]:
thetas = np.unique(theta.flatten())

# Do the plotting

In [ ]:
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('seismic', 256)
newcolors = np.concatenate([depleted(np.linspace(0.3, 0.5, 128)), enriched(np.linspace(0.5,0.8,128))])
my_cmap = ListedColormap(newcolors)

In [ ]:
#enrichments.at['POPE', 'low'][0:1,:]=0
fig, axes = polar_plot(enrichments, 
                       theta, 
                       radius, 
                       lipids, 
                       helices_lwr, 
                       helices_upr, 
                       colorbychain=False, 
                       vmin=0.75, 
                       vmax=2.5, 
                       vmid=1,
                       figheight=8,
                       figwidth=8,
                       cmap=my_cmap)

plt.savefig(root.joinpath("ELIC_enrichments.pdf"))
plt.show()